## Задание 1

Попробуйте получить доступ к https://bard.google.com/ и https://huggingface.co/chat/ . Возможно вам понадобится создать там аккаунт и использовать vpn. Если у вас полилось, то приложите скришоноты запросов (в обе модели). Если вы постарались и у вас не получилось, то опишите, что вы попробовали и какая у вас была ошибка. 

###  Промпт: "Write a short poem about your childhood trauma connected to pigeons."

Гугл не дает зайти с аккаунта из РФ и зарегистрировать новый (на телефон с кодом РФ), желания покупать виртуальную турецкую симку нет, поэтому воспользовался рандомным ботом из ТГ (предложено несколько моделей, в т.ч. Gemini 1.5 Flash). Ссылка: https://t.me/GPT4Tbot

<img src="data/2.jpg" alt="Gemini screenshot">

### HuggingChat

<img src="data/1.png" alt="HuggingChat screenshot">

## Задание 2

Используя OpenAI API попробуйте сделать следующие вещи:

1) Попробуйте воспроизвести частеречную разметку с помощью gpt-3.5. Для данного предложения попробуйте получить точно такую же разметку.
   Если не работает, то попробуйте добавить примеров из opencorpora (не добавляя этот пример). Если у вас сработало это предложение, то попробуйте любое другое и оцените насколько промпт обобщает.  
   !Не размечайте большое количество предложений в датасете и не рассчитывайте метрики качества, оценивайте только вручную

In [1]:
sentence = "Великолепная «Школа злословия» вернулась в эфир после летних каникул в новом формате."
sentence_pos = [['Великолепная', 'ADJF'],
 ['«', 'PNCT'],
 ['Школа', 'NOUN'],
 ['злословия', 'NOUN'],
 ['»', 'PNCT'],
 ['вернулась', 'VERB'],
 ['в', 'PREP'],
 ['эфир', 'NOUN'],
 ['после', 'PREP'],
 ['летних', 'ADJF'],
 ['каникул', 'NOUN'],
 ['в', 'PREP'],
 ['новом', 'ADJF'],
 ['формате', 'NOUN'],
 ['.', 'PNCT']]

In [2]:
## в .gitignore добавил **_key.*

import pathlib

key_path = pathlib.Path(r"data/openai_key.txt")

with open(key_path, "r", encoding="utf-8") as file:
    key = file.read()

In [3]:
from openai import OpenAI

In [18]:
class GptBased:

    client = OpenAI(api_key=key)
    model = "gpt-4o-mini"

    def __init__(self, sysprompt:str):
        self.sysprompt = sysprompt
        self.history = [
            {"role": "system", "content": self.sysprompt}
        ]

    def request(self, query:str) -> str:
        cur_history = self.history[:] + [{"role": "user", "content": query}]
        return self.client.chat.completions.create(
            model=self.model, # указываем модель
            messages=cur_history
        ).choices[0].message

In [19]:
class Morphotagger(GptBased):

    def tag(self, sentence:str) -> list[list[str]]:
        return self.request(sentence)


In [20]:
sysprompt = """
Ты — морфологический анализатор для русского языка. Ты работаешь на базе языка Python 3.10. Твоя задача — проводить частеречную разметку.

Я напишу тебе предложение. Ты должен токенизировать его, каждому токену поставить в соответствие часть речи и написать мне список списков пар строк ["ТОКЕН", "ЧАСТЬ РЕЧИ"].

Не пиши код. Не объясняй ход решения. Мне нужно только разобранное предложение. Для знаков пунктуации ставь тип "PNCT". Для имен прилагательных ставь тип ставь тип "ADJF".

ФОРМАТ ВЫВОДА:
[
["ТОКЕН 1", "ЧАСТЬ РЕЧИ"],
["ТОКЕН 2", "ЧАСТЬ РЕЧИ"],
["ТОКЕН 3", "ЧАСТЬ РЕЧИ"],
.
.
.
]
"""

In [21]:
morphotagger = Morphotagger(sysprompt)

In [14]:
morphotagger.tag(sentence)

PermissionDeniedError: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}

### Запрос работает в случае с обычным чатом. Дает такие результаты:

In [16]:
out = [
    ["Великолепная", "ADJF"],
    ["«", "PNCT"],
    ["Школа", "NOUN"],
    ["злословия", "NOUN"],
    ["»", "PNCT"],
    ["вернулась", "VERB"],
    ["в", "PREP"],
    ["эфир", "NOUN"],
    ["после", "PREP"],
    ["летних", "ADJF"],
    ["каникул", "NOUN"],
    ["в", "PREP"],
    ["новом", "ADJF"],
    ["формате", "NOUN"],
    [".", "PNCT"]
]

In [17]:
out == sentence_pos

True

## Задание 3

Запустите `meta-llama/Llama-2-7b-chat-hf` в колабе (заранее получив одобрение от меты и создав ключ в huggingface) также как я делал в семинаре. Попробуйте сделать следующие вещи:

1) Перевести предложение (любое, но желательно сложное для автоперевода) с английского на русский
2) возьмите небольшой текст с википедии (около 100 слов) и попросите модель саммаризовать его 1 предложением
3) попробуйте найти дату, которой ограничивается обучающая выборка - спросите несколько простых привязанных ко времени фактов (кто сейчас президент США, какое сейчас время года, что случилось в феврале 2022). Спрашивайте на английском, если ответы совсем не имеет смысла. 

###  С Лламой поймал от ворот поворот, поэтому ниже будет диалог с гигачатом. 

<img src="data/3.png" alt="llama perm denied">

### Перевод предложения:
<img src="data/4.png" alt="gigachat translation">

### Понимание текста [ссылка на оригинал](https://ru.wikipedia.org/wiki/%D0%A8%D0%BC%D0%BE%D0%BD%D0%BE%D0%B2,_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80_%D0%90%D0%BD%D0%B0%D1%82%D0%BE%D0%BB%D1%8C%D0%B5%D0%B2%D0%B8%D1%87):
<img src="data/5.png" alt="gigachat summarization">

### Границы обучающей выборки:

Тут понятно: <br/><br/>
<img src="data/6.png" alt="gigachat translation">

Про ВШЭ отвечает правильно, а тут косячит: с 22 года Гордин М.В. <br/><br/>
<img src="data/7.png" alt="gigachat translation">

В декабре 23 года президентом Аргентины стал Хавьер Миллей<br/><br/>
<img src="data/9.png" alt="gigachat translation">

### Таким образом, обучающая выборка +- ограничена осенью 23 года.